In [1]:
from kiwipiepy import Kiwi
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from datetime import date
import os

tqdm.pandas()

In [2]:
kiwi = Kiwi(model_type='sbg',num_workers=0)

In [4]:
df = pd.read_csv('/home/intern2/fire_dev/NEWS_NLP/ES.csv',encoding='utf-8-sig')

In [3]:
df = pd.read_csv('cleansing.csv',encoding='utf-8-sig')

KeyboardInterrupt: 

In [5]:
df

,YNewsML.NewsContent.Title,YNewsML.NewsContent.Body,send_timestamp
0,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편,신분당선 판교∼정자역 사이서 전동차 멈춰…1시간가량 불편\n \r\n (...,2022-12-31T23:20:59
1,남아공 가스트럭 폭발사고 사망자 34명으로 늘어,남아공 가스트럭 폭발사고 사망자 34명으로 늘어\n\r\n<YNAOBJECT id=...,2022-12-31T18:08:13
2,"선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)","선로에 떨어진 부직포 때문에…SRT, 운행 차질에 차량 고장까지(종합)\n전날 선로...",2022-12-31T16:16:20
3,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소,선로 전기공급 끊기고 SRT 고장…이틀째 대거 운행지연·취소\n첫날 KTX·SRT ...,2022-12-31T12:57:34
4,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료,캄보디아 카지노 호텔 화재 사망자 최소 27명으로…수색 종료\n\r\n<YNAOBJ...,2022-12-31T11:51:25
...,...,...,...
11052,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인","한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인\n\r\n<YNAOBJ...",2022-06-08T10:33:09
11053,"소방청-가스안전공사, 신에너지 사고 대응 업무협약","소방청-가스안전공사, 신에너지 사고 대응 업무협약\n\r\n<YNAOBJECT id...",2022-06-08T10:00:07
11054,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축","LGU+, 서울지하철 8호선에 '스마트 역사' 구축\r\n (서울=연합뉴스) ...",2022-06-08T09:20:49
11055,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료","LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료\n역사 내 사고에 신속 대...",2022-06-08T09:00:02


In [6]:
df = df.dropna().reset_index().drop(columns='index',axis=1)

In [13]:
import re

In [14]:
def cleansing(body, mode=None):
    if type(body) == np.ndarray:
        body = np.round(body.tolist(), 5)
        body = body.tolist()
        return body
    else:
        body = re.sub('<YNAOBJECT.*?/YNAOBJECT>', '', body, 0, re.I|re.S) # YNAOBJECT 태그 제거
        body = re.sub('<table.*?/table>', '', body, 0, re.I|re.S) # table 태그 제거
        
        if mode == 'meta':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[0]:]
            except AttributeError:
                pass
            return body
        
        if mode == 'sum':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[1]:]
            except AttributeError:
                pass
            
        
        body = re.sub(r'\([^)]*\)', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\[.*\]|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\<.*\>|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub('·', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('-', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('\(.*?연합뉴스\).*?=', '', body, 0, re.I|re.S) # 기자명 및 기자소속 제거
        body = re.sub('\(서울.*?연합뉴스\)', '', body, 0, re.I|re.S) # 기자소속 제거
        body = re.sub('yna.co.kr', '', body, 0, re.I|re.S) # 연합 url 제거
        body = re.sub('([a-zA-Z0-9\_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', body, 0, re.I|re.S) # E-mail 제거
        body = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', body, 0, re.I|re.S) # URL 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+억원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천만달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+만원)', '', body, 0, re.I|re.S) # 만 제거
#         body = re.sub('([0-9\_.+-]+만[0-9-]천[0-9-]+원)', '', body, 0, re.I|re.S) # 만 제거
        body = re.sub('전문보기.*?\n', '', body, 0, re.I|re.S) # 전문보기 제거
        body = re.sub('&lt;.*?&gt;&gt;', '', body, 0, re.I|re.S) # 기타 안내문 제거
        body = re.sub('&lt;.*?&gt;', '', body, 0, re.I|re.S) # '&gt;'가 1개인 안내문 제거
        body = re.sub('&apos;', "'", body, 0, re.I|re.S) # &apos; ' 변환
        body = re.sub('&amp;', "&", body, 0, re.I|re.S) # &amp;; ' 변환        
        body = re.sub('-.*?_.*?\(끝\)', '', body, 0, re.I|re.S) # 9001887 속성에만 존재하는 불용어 제거
        body = re.sub('[#@*※|》■▲▼◀▶◇☎]-<>=+·↑', '', body, 0, re.I|re.S) # 특수문자 제거
        body = re.sub('…', '', body, 0, re.I|re.S) # … 제거
        body = re.sub('\(끝\)', '', body, 0, re.I|re.S) # 끝 태그 제거
        body = re.sub('[""]', '', body, 0, re.I|re.S) # "" 제거
        body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거


        paragraphs = body.split('\r\n') # 단락 분리
        if ' 기자' in paragraphs[-1]: del paragraphs[-1] # 끝 단락 기자명 제거
        body = []
        for paragraph in paragraphs:
            paragraph = re.sub('\[.+?\]', '', paragraph, 0, re.I|re.S)
            paragraph = ' '.join(paragraph.split()) # 문자열 중간 다중 공백 제거
            if len(paragraph) > 0:
                body.append(paragraph)
        
        if mode == 'quot':
            return body
        else:
            return ' '.join(body)

In [36]:
clean = df['YNewsML.NewsContent.Title'].apply(cleansing)

In [37]:
clean

0           신분당선 판교 정자역 사이서 전동차 멈춰 시간가량 불편
1                 남아공 가스트럭 폭발사고 사망자 명으로 늘어
2        선로에 떨어진 부직포 때문에SRT 운행 차질에 차량 고장까지
3         선로 전기공급 끊기고 SRT 고장이틀째 대거 운행지연 취소
4           캄보디아 카지노 호텔 화재 사망자 최소 명으로수색 종료
                       ...                
11052     한국소비자원 편의점 업계와 여름철 먹거리 안전 관리 캠페인
11053           소방청 가스안전공사 신에너지 사고 대응 업무협약
11054             LGU+ 서울지하철 호선에 스마트 역사 구축
11055          LGU+ 서울지하철 호선에 스마트 역사 구축 완료
11056          제조업 운반 하역 작업서 부상 가장 많이 발생전체
Name: YNewsML.NewsContent.Title, Length: 11057, dtype: object

In [38]:
f = open(f'es_dict.txt','w',encoding='utf-8-sig')

for i in clean:
    f.write(str(i))
f.close()

In [39]:
inputs = list(open(f'es_dict.txt', encoding='utf-8-sig'))
result = kiwi.extract_words(inputs, min_cnt=1, max_word_len=10, min_score=0.15)

In [41]:
pd.DataFrame(result).to_csv('new_word_es.csv',encoding='utf-')

,0,1,2,3
0,서울소방재난본부장,0.871961,27,-1.754715
1,안전관리,0.864604,200,-2.905137
2,중대재해,0.784491,126,-0.984302
3,정례브리핑,0.727176,23,-0.355426
4,DL이앤씨,0.674691,5,-2.757214
5,용산소방서장,0.518236,26,-2.243024
6,HDC현,0.467727,10,-0.116693
7,기후변화,0.434579,22,-2.512540
8,안전보건,0.430151,49,-1.697576
9,경부고속,0.427345,17,-0.657891


In [ ]:
def month(data):
    data = pd.to_datetime(str(data))
    return data.month

In [ ]:
def year(data):
    data = pd.to_datetime(str(data))
    return data.year

In [ ]:
def day(data):
    data = pd.to_datetime(str(data))
    return data.day

In [ ]:
from datetime import datetime, timedelta
def get_date(date):
    return pd.to_datetime(date)

In [ ]:
def get_week_no(date):
    target = get_date(date)
    firstday = target.replace(day=1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days=6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [ ]:
df['month'] = df.Date.apply(month)

In [ ]:
df['year'] = df.Date.apply(year)

In [ ]:
df['day'] = df.Date.apply(day)

In [ ]:
df['week'] = df.Date.apply(get_week_no)

In [ ]:
df.to_csv('cleansing0.csv', encoding='utf-8-sig', index=False)

In [3]:
df = pd.read_csv('cleansing0.csv', encoding='utf-8-sig')

In [13]:
def kspace(data):
    result = kiwi.space(data,reset_whitespace=True)
    return result

In [14]:
df.TB[:300000] = df.TB[:300000].progress_apply(kspace)

100%|██████████| 300000/300000 [1:28:31<00:00, 56.48it/s]  
/tmp/ipykernel_20758/183453959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.TB[:300000] = df.TB[:300000].progress_apply(kspace)


In [15]:
df.TB[300000:633369] = df.TB[300000:633369].progress_apply(kspace)

100%|██████████| 333369/333369 [1:41:34<00:00, 54.70it/s]  
/tmp/ipykernel_20758/1428459661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.TB[300000:633369] = df.TB[300000:633369].progress_apply(kspace)


In [ ]:
for k in range(2021,2023):
    for j in range(1,13):
        cond1 = df.year == k  
        cond2 = df.month == j
        f = open(f'new_dict{k}_{j}.txt','w',encoding='utf-8-sig')

        for i in df.loc[cond1&cond2].TB:
            f.write(str(i))
        f.close()

In [20]:
for k in range(2021,2023):
    for i in range(1,13):
        inputs = list(open(f'new_dict{k}_{i}.txt', encoding='utf-8-sig'))
        globals()[f"result{k}_{i}"] = kiwi.extract_words(inputs, min_cnt=10, max_word_len=10, min_score=0.25)

In [21]:
result1 = pd.DataFrame()

In [32]:
for k in range(2021,2023):
    for i in range(1,11):
            result1 = pd.concat([result1,pd.DataFrame((globals()[f"result{k}_{i}"]))]).drop_duplicates(subset=0)
            os.remove(f'new_dict{k}_{i}.txt')

In [33]:
result1

,0,1,2,3
0,동부구치소,0.696736,892,-1.747690
1,아동보호전문기관,0.669614,75,-2.749990
2,탄도미사일,0.606798,125,-2.900636
3,거시경제금융회의,0.438067,30,-2.585476
4,선별검사소,0.422928,475,-0.424993
...,...,...,...,...
30,산창조경제혁신센터,0.288560,13,-1.408146
37,아시아문화전당,0.256280,26,-2.156924
38,립교향악단,0.255154,24,-2.510122
39,국가산업단지,0.251288,46,-1.827210


In [34]:
result1.to_csv('result_month_TB.csv',encoding='utf-8-sig', index=False)

In [376]:
for i in range(1,13):
    for j in range(1,7):
        try:
            cond1 = df.year == 2021  
            cond2 = df.month == i
            cond3 = df.week == j

            f = open(f'new_dict{i}.{j}week.txt','w',encoding='utf-8-sig')

            for k in df.loc[cond1&cond2&cond3].clean:
                    f.write(str(k))
            f.close()
        except:
            pass

In [377]:
for i in range(1,13):
    for j in range(1,7):
        try:
            inputs = list(open(f'new_dict{i}.{j}week.txt', encoding='utf-8-sig'))
            globals()[f"result{i}_{j}"] = kiwi.extract_words(inputs, min_cnt=10, max_word_len=10, min_score=0.25)
        except:
            pass

In [384]:
result = pd.DataFrame()

In [385]:
for i in range(1,13):
    for j in range(1,7):
        try:
            result = pd.concat([result,pd.DataFrame((globals()[f"result{i}_{j}"]))]).drop_duplicates(subset=0)
        except:
            pass

In [387]:
result.to_csv('result_week_body.csv',encoding='utf-8-sig', index=False)

In [16]:
for p in range(2021,2023):
    for i in range(1,13):
        for j in range(1,32):
                try:
                    cond1 = df.year == p  
                    cond2 = df.month == i
                    cond3 = df.day == j
                    f = open(f'new_dict{p}_{i}_{j}_7.txt','w',encoding='utf-8-sig')

                    for k in df.loc[cond1&cond2&cond3].TB:
                        f.write(str(k))
                    f.close()
                except:
                    pass

In [17]:
for p in range(2021,2023):    
    for i in tqdm(range(1,13)):
        for j in range(1,32):
                    try:
                        inputs = list(open(f'new_dict{p}_{i}_{j}_7.txt', encoding='utf-8-sig'))
                        globals()[f"result{p}_{i}_{j}_7"] = kiwi.extract_words(inputs, min_cnt=7, max_word_len=10, min_score=0.25)
                    except:
                        pass

100%|██████████| 12/12 [05:01<00:00, 25.15s/it]


In [18]:
result5 = pd.DataFrame()
result7 = pd.DataFrame()
result9 = pd.DataFrame()

In [19]:
for p in tqdm(range(2021,2023)):       
    for i in tqdm(range(1,13)):
        for j in range(1,32):
                try:
                        result7 = pd.concat([result7,pd.DataFrame((globals()[f"result{p}_{i}_{j}_7"]))]).drop_duplicates(subset=0)
                except:
                    pass
                os.remove(f'new_dict{p}_{i}_{j}_7.txt')

100%|██████████| 2/2 [00:00<00:00,  3.18it/s]


In [20]:
result7

,0,1,2,3
0,장직무대리,0.870138,13,-2.545420
0,허베이성,0.638081,12,-0.141931
1,스자좡,0.522854,18,-0.702574
0,렉키로나주,1.434822,16,-0.067069
1,다셀트리온,0.596039,15,-2.053973
...,...,...,...,...
1,경윳값,0.253197,7,0.376209
0,도교육청,0.322942,17,-1.946080
0,머티리얼즈,0.383598,11,-2.155141
0,CJ프레시웨이,0.326998,8,-1.876121


In [9]:
result5.to_csv('result_day_TB_5.csv',encoding='utf-8-sig', index=False)

In [21]:
result7.to_csv('result_day_TB_7_space.csv',encoding='utf-8-sig', index=False)

In [11]:
result9.to_csv('result_day_TB_9.csv',encoding='utf-8-sig', index=False)

In [9]:
inputs = list(open('new_dict.txt', encoding='utf-8-sig'))
result = kiwi.extract_words(inputs, min_cnt=10, max_word_len=10, min_score=0.25)

In [ ]:
len(result)

792

In [ ]:
f = open('new_dict1.txt','w',encoding='utf-8-sig')

In [ ]:
for i in df.title:
    f.write(str(i))
f.close

<function TextIOWrapper.close()>

In [ ]:
inputs = list(open('new_dict1.txt', encoding='utf-8-sig'))
result1 = kiwi.extract_words(inputs, min_cnt=10, max_word_len=10, min_score=0.25)

In [ ]:
len(result1)

891

In [ ]:
f = open('new_dict2.txt','w',encoding='utf-8-sig')

In [ ]:
for i in df.TB:
    f.write(str(i))
f.close

<function TextIOWrapper.close()>

In [ ]:
inputs = list(open('new_dict2.txt', encoding='utf-8-sig'))
result2 = kiwi.extract_words(inputs, min_cnt=10, max_word_len=10, min_score=0.25)

In [ ]:
len(result2)

715

In [ ]:
result = pd.DataFrame(result).rename(columns={0:'word',1:'FinScore',2:'counts',3:'score'})

In [ ]:
result.to_csv('newword_body1.csv',encoding='utf-8-sig', index=False)

In [ ]:
result1 = pd.DataFrame(result1).rename(columns={0:'word',1:'FinScore',2:'counts',3:'score'})

In [ ]:
result1.to_csv('newword_title1.csv',encoding='utf-8-sig', index=False)

In [ ]:
result2 = pd.DataFrame(result2).rename(columns={0:'word',1:'FinScore',2:'counts',3:'score'})

In [ ]:
result2.to_csv('newword_TB1.csv',encoding='utf-8-sig', index=False)

In [63]:
result_month = pd.read_csv('result_day_TB_5.csv',encoding='utf-8-sig')

result_month_body = pd.read_csv('result_day_TB_7.csv',encoding='utf-8-sig')

result1 = pd.read_csv('result_day_TB_9.csv',encoding='utf-8-sig')